In [1]:
#! /usr/bin/env python
"""
Compute debris thickness through sub-debris and temperature inversion methods
"""
import sys
import os
import re
import subprocess
from datetime import datetime, timedelta
import time
import pickle
from collections import OrderedDict

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio
from rasterio.merge import merge
from rasterio.warp import calculate_default_transform, reproject, Resampling
from scipy import ndimage
from scipy.optimize import curve_fit
from scipy.optimize import minimize
from scipy.stats import median_absolute_deviation
import xarray as xr
from osgeo import gdal, ogr, osr

from pygeotools.lib import malib, warplib, geolib, iolib, timelib


import debrisglobal.globaldebris_input as debris_prms
from debrisglobal.glacfeat import GlacFeat, create_glacfeat
from meltcurves import melt_fromdebris_func
from meltcurves import debris_frommelt_func
from spc_split_lists import split_list


debug=False

In [2]:
#Function to generate a 3-panel plot for input arrays
def plot_array(dem, clim=None, titles=None, cmap='inferno', label=None, overlay=None, fn=None, close_fig=True):
    fig, ax = plt.subplots(1,1, sharex=True, sharey=True, figsize=(10,5))
    alpha = 1.0
    #Gray background
    ax.set_facecolor('0.5')
    #Force aspect ratio to match images
    ax.set(aspect='equal')
    #Turn off axes labels/ticks
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    if titles is not None:
        ax.set_title(titles[0])
    #Plot background shaded relief map
    if overlay is not None:
        alpha = 0.7
        ax.imshow(overlay, cmap='gray', clim=(1,255))
    #Plot each array
    im_list = [ax.imshow(dem, clim=clim, cmap=cmap, alpha=alpha)]
    fig.tight_layout()
    fig.colorbar(im_list[0], label=label, extend='both', shrink=0.5)
    if fn is not None:
        fig.savefig(fn, bbox_inches='tight', pad_inches=0, dpi=150)
    if close_fig:
        plt.close(fig)
        
        
def maskedarray_gt(data, value):
    """ Greater than operation on masked array to avoid warning errors """
    data = np.nan_to_num(data,0)
    data[data > value] = value
    return data


def maskedarray_lt(data, value):
    """ Less than operation on masked array to avoid warning errors """
    data = np.nan_to_num(data,0)
    data[data < value] = value
    return data


def ts_fromdebris_func(h, a, b, c):
    """ estimate surface temperature from debris thickness (h is debris thickness, a and k are coefficients) 
        Hill Equation"""
    return a * h**c / (b**c + h**c)


def debris_fromts_func(ts, a, b, c, hd_max=debris_prms.hd_max):
    """ estimate debris thickness from surface temperature (ts is surface temperature, a and k are coefficients) 
        Hill Equation"""
    # If temperature exceeds maximum of function cause NaN value
    max_value = ts_fromdebris_func(50, a, b, c)
    if ts.size == 1:
        if ts > max_value:
            ts = max_value
        if ts < 0:
            ts = 0
    else:
        ts[ts > a] = max_value
        ts[ts < 0] = 0
    # Debris thickness
    hd = (ts * b**c / (a - ts))**(1/c)
    return hd
        
    
def debris_fromts_maskedarray(ts_raw, a, b, c):
    """ Apply debris_fromts_func to masked array
        includes a mask of maximum values, since Michaelis-Mentin Equation has natural maximum 
    Parameters
    ----------
    ts_raw : np.ma.array
        masked array of the unmodified surface temperature
    Returns
    -------
    hd : np.ma.array 
        masked array of the debris thickness (m)
    """
    hd = debris_fromts_func(ts_raw.data, a, b, c)
    return hd

In [3]:
# ===== DETERMINE ALL GLACIERS WITH AND WITHOUT OBSERVATIONS =====
hdopt_prms_fp = debris_prms.output_fp + 'hd_opt_prms/' + debris_prms.roi + '/'

# Glaciers optimized
glac_tsopt_fns = []
rgiid_list_tsopt = []
for roi_extrap in debris_prms.roi_dict_extrap[debris_prms.roi]:
    hdopt_prms_fp = debris_prms.output_fp + 'hd_opt_prms/' + roi_extrap + '/'
    for i in os.listdir(hdopt_prms_fp):
        if i.endswith('_hdopt_prms.csv'):
            region = int(i.split('.')[0])
            if region in debris_prms.roi_rgidict[roi_extrap]:    
                rgiid_list_tsopt.append(i.split('_')[0])            
                glac_tsopt_fns.append(i)
    glac_tsopt_fns = sorted(glac_tsopt_fns)
    rgiid_list_tsopt = sorted(rgiid_list_tsopt)

# if debris_prms.roi == '16':
#     hdopt_prms_fp2 = debris_prms.output_fp + 'hd_opt_prms/17/'
#     for i in os.listdir(hdopt_prms_fp2):
#         if i.endswith('_hdopt_prms.csv'):
#             region = int(i.split('.')[0])
#             if region in [17]:    
#                 rgiid_list_tsopt.append(i.split('_')[0])            
#                 glac_tsopt_fns.append(i)
#     glac_tsopt_fns = sorted(glac_tsopt_fns)
#     rgiid_list_tsopt = sorted(rgiid_list_tsopt)
    

main_glac_rgi_tsopt = debris_prms.selectglaciersrgitable(rgiid_list_tsopt)

# All debris-covered glaciers
dc_shp = gpd.read_file(debris_prms.debriscover_fp + debris_prms.debriscover_fn_dict[debris_prms.roi])
dc_rgiid = sorted([x.split('-')[1] for x in dc_shp.RGIId])
main_glac_rgi_all = debris_prms.selectglaciersrgitable(glac_no=dc_rgiid)

# Merge with debris cover stats
dc_shp = gpd.read_file(debris_prms.debriscover_fp + debris_prms.debriscover_fn_dict[debris_prms.roi])
dc_shp = dc_shp.sort_values(by=['RGIId'])
dc_shp.reset_index(inplace=True, drop=True)

# Add debris stats to area
dc_areaperc_dict = dict(zip(dc_shp.RGIId.values,dc_shp['DC_Area__1'].values))
dc_area_dict = dict(zip(dc_shp.RGIId.values,dc_shp['DC_Area_v2'].values))

main_glac_rgi_tsopt['DC_Area_%'] = main_glac_rgi_tsopt.RGIId.map(dc_areaperc_dict).fillna(0)
main_glac_rgi_all['DC_Area_%'] = main_glac_rgi_all.RGIId.map(dc_areaperc_dict).fillna(0)
main_glac_rgi_tsopt['DC_Area_v2'] = main_glac_rgi_tsopt['Area'] * main_glac_rgi_tsopt['DC_Area_%'] / 100
main_glac_rgi_all['DC_Area_v2'] = main_glac_rgi_all['Area'] * main_glac_rgi_all['DC_Area_%'] / 100

# Glaciers lacking optimization
rgiids_missing = set(main_glac_rgi_all.rgino_str.values) - set(main_glac_rgi_tsopt.rgino_str.values)
rgiids_missing = sorted(rgiids_missing)
main_glac_rgi_missing = debris_prms.selectglaciersrgitable(rgiids_missing)
main_glac_rgi_missing

234 glaciers in region 17 are included in this model run: ['00516', '01019', '01448', '01506', '01687', '01917', '01940', '01972', '02025', '02427', '02600', '02698', '02737', '02806', '02857', '03289', '03291', '03301', '03309', '03311', '03318', '03325', '03348', '03353', '03359', '03371', '03406', '03418', '03424', '03425', '03451', '03455', '03469', '03519', '03531', '03545', '03610', '03645', '03646', '03655', '03750', '03899', '03903', '03925', '03927', '03929', '04065', '04101', '04108', '04110'] and more
This study is focusing on 234 glaciers in region [17]
6074 glaciers in region 17 are included in this model run: ['00001', '00008', '00009', '00011', '00013', '00014', '00015', '00016', '00021', '00022', '00023', '00024', '00029', '00031', '00032', '00055', '00064', '00065', '00066', '00067', '00068', '00070', '00074', '00080', '00081', '00083', '00095', '00102', '00122', '00134', '00136', '00141', '00144', '00148', '00172', '00271', '00312', '00367', '00368', '00396', '00412',

,O1Index,RGIId,CenLon,CenLat,O1Region,O2Region,Area,Zmin,Zmax,Zmed,Slope,Aspect,Lmax,Form,TermType,Surging,RefDate,glacno,rgino_str,RGIId_float
GlacNo,,,,,,,,,,,,,,,,,,,,
0,0,RGI60-17.00001,-70.2602,-34.4356,17,2,0.888,2966,3571,3242,22.4,203,1766,0,0,9,20019999,1,17.00001,17.00001
1,7,RGI60-17.00008,-70.3739,-34.5810,17,2,0.241,3537,4121,3847,33.2,78,807,0,0,9,20019999,8,17.00008,17.00008
2,8,RGI60-17.00009,-70.2728,-34.5863,17,2,0.158,3387,3586,3472,33.8,107,280,0,0,9,20019999,9,17.00009,17.00009
3,10,RGI60-17.00011,-70.3864,-34.5902,17,2,1.107,3106,4124,3562,26.1,228,1819,0,0,9,20019999,11,17.00011,17.00011
4,12,RGI60-17.00013,-70.2570,-34.5947,17,2,0.510,3121,3464,3244,22.2,166,834,0,0,9,20019999,13,17.00013,17.00013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5835,15902,RGI60-17.15903,-73.6700,-47.0460,17,1,161.389,16,2401,945,10.8,272,31214,0,2,9,20011103,15903,17.15903,17.15903
5836,15903,RGI60-17.15904,-73.6650,-46.9760,17,1,31.033,68,2192,1274,14.1,286,13890,0,2,9,20011103,15904,17.15904,17.15904
5837,15904,RGI60-17.15905,-73.6750,-46.9340,17,1,16.863,424,1614,1130,13.1,252,6356,0,0,9,20011103,15905,17.15905,17.15905


In [4]:
print('\n', debris_prms.roi + ': calibration includes', main_glac_rgi_tsopt.shape[0], 'glaciers covering',
      str(np.round(main_glac_rgi_tsopt['DC_Area_v2'].sum(),1)), 'km2 (' + 
      str(np.round(main_glac_rgi_tsopt['DC_Area_v2'].sum() / main_glac_rgi_all['DC_Area_v2'].sum() * 100,1)) + 
      '%) of the total debris-covered glacier area\n')     

print('  glaciers all:', main_glac_rgi_all['DC_Area_v2'].shape[0], 
      'total dc area [km2]:', np.round(main_glac_rgi_all['DC_Area_v2'].sum(),1))


 17: calibration includes 234 glaciers covering 468.3 km2 (24.8%) of the total debris-covered glacier area

  glaciers all: 6074 total dc area [km2]: 1889.7


In [5]:
# # print('\nHACK TO EXTRAPOLATE TO CALIBRATED GLACIERS FOR COMPARISON\n')
# rgiids_missing = ['15.02167']
# # # rgiids_missing = ['2.14297']
# # # rgiids_missing = ['11.02472']
# # # rgiids_missing = ['12.01132']
# # # rgiids_missing = ['13.43165']
# # # rgiids_missing = ['15.03473']
# # # rgiids_missing = ['15.04045']
# # # rgiids_missing = ['18.02397']
# # rgiids_missing = ['11.03206']

# main_glac_rgi_missing = debris_prms.selectglaciersrgitable(rgiids_missing)

In [7]:
extrap_uncalibrated_glaciers = True
overwrite_hd = True

hd_fp = debris_prms.hd_fp + 'extrap/'
if not os.path.exists(hd_fp):
    os.makedirs(hd_fp)
    
mf_fp = hd_fp + 'meltfactor/'
if not os.path.exists(mf_fp):
    os.makedirs(mf_fp)
    
fig_extrap = debris_prms.output_fig_fp + debris_prms.roi + '/' + 'extrap/'
if not os.path.exists(fig_extrap):
    os.makedirs(fig_extrap)

if extrap_uncalibrated_glaciers:
    
    # ===== NEAREST GLACIERS WITH DATA =====
    n_glac_nearest = 1000
    if n_glac_nearest > main_glac_rgi_tsopt.shape[0]:
        n_glac_nearest = main_glac_rgi_tsopt.shape[0]

    nearest_dict = {}
    for nglac, glac_idx in enumerate(main_glac_rgi_missing.index.values):
#     for nglac, glac_idx in enumerate([main_glac_rgi_missing.index.values[0]]):
        glac_str = main_glac_rgi_missing.loc[glac_idx, 'rgino_str']
        if glac_idx%500 == 0:
            print(glac_idx, glac_str)
        latlon_dist = (((main_glac_rgi_tsopt['CenLat'].values - main_glac_rgi_missing['CenLat'].values[glac_idx])**2 + 
                        (main_glac_rgi_tsopt['CenLon'].values - main_glac_rgi_missing['CenLon'].values[glac_idx])**2)**0.5)

        latlon_nearidx_list = np.argsort(latlon_dist)[0:n_glac_nearest]
        rgiid_nearest_list = list(main_glac_rgi_tsopt.loc[latlon_nearidx_list,'rgino_str'].values)

        nearest_dict[glac_str] = rgiid_nearest_list

    # Ts filenames
    ts_fns_df = pd.read_csv(debris_prms.ts_fp + debris_prms.ts_fns_fn)
    
    for nglac, glac_idx in enumerate(main_glac_rgi_missing.index.values):
#     for nglac, glac_idx in enumerate(main_glac_rgi_missing.index.values[4290:]):
#     for nglac, glac_idx in enumerate([main_glac_rgi_missing.index.values[0]]):
        glac_str = main_glac_rgi_missing.loc[glac_idx, 'rgino_str']
        rgiid = main_glac_rgi_missing.loc[glac_idx,'RGIId']
        region = glac_str.split('.')[0]

        if int(region) < 10:
            glac_str_noleadzero = str(int(glac_str.split('.')[0])) + '.' + glac_str.split('.')[1]
        else:
            glac_str_noleadzero = glac_str

        # Ts filename
        ts_fn_idx = np.where(ts_fns_df['RGIId'].values == rgiid)[0][0]
        ts_fn = ts_fns_df.loc[ts_fn_idx,'ts_fullfn']
        
        # Hd filename
        hd_fn = debris_prms.hd_fn_sample.replace('XXXX', glac_str_noleadzero).replace('.tif','_extrap.tif')

        # Ice thickness filenames
        thick_dir = debris_prms.oggm_fp + 'thickness/RGI60-' + str(region.zfill(2)) + '/'
        thick_fn = 'RGI60-' + str(region.zfill(2)) + '.' + rgiid.split('.')[1] + '_thickness.tif'

        # Record values from nearest for the melt factors after debris thickness is extrapolated
        melt_2cm_nearest = None
        melt_cleanice_nearest = None
        func_coeff_nearest = None
        
        if ((not os.path.exists(hd_fp + hd_fn) or overwrite_hd) and os.path.exists(thick_dir + thick_fn) and 
            ts_fn not in ['0.0']):
            
            print(glac_idx, glac_str)
            
            # Create glacier feature
            gf = create_glacfeat(thick_dir, thick_fn)
        
            # Debris shape layer processing
            dc_shp_proj_fn = (debris_prms.glac_shp_proj_fp + glac_str + '_dc_crs' + 
                              str(gf.aea_srs.GetAttrValue("AUTHORITY", 1)) + '.shp')
            if not os.path.exists(dc_shp_proj_fn):
                dc_shp_init = gpd.read_file(debris_prms.debriscover_fp + 
                                            debris_prms.debriscover_fn_dict[debris_prms.roi])
                dc_shp_single = dc_shp_init[dc_shp_init['RGIId'] == rgiid]
                dc_shp_single = dc_shp_single.reset_index()
                dc_shp_proj = dc_shp_single.to_crs({'init': 'epsg:' + 
                                                    str(gf.aea_srs.GetAttrValue("AUTHORITY", 1))})
                dc_shp_proj.to_file(dc_shp_proj_fn)
            dc_shp_ds = ogr.Open(dc_shp_proj_fn, 0)
            dc_shp_lyr = dc_shp_ds.GetLayer()

            # Add layers
            gf.add_layers(dc_shp_lyr, gf_add_ts=True, ts_fn=ts_fn, gf_add_slope_aspect=False)

#             # ===== PLOTS =====
#             if debug:
#                 # DEM
#                 var_full2plot = gf.z1.copy()
#                 clim = malib.calcperc(var_full2plot, (2,98))
#                 plot_array(var_full2plot, clim, [glac_str + ' DEM'], 'inferno', 'elev (masl)', close_fig=False)
#                 # Surface temperature
#                 var_full2plot = gf.ts.copy()
#                 clim = malib.calcperc(var_full2plot, (2,98))
#                 plot_array(var_full2plot, clim, [glac_str + ' Ts'], 'inferno', 'ts (degC)', close_fig=False)
#                 # Surface temperature (debris-covered)
#                 var_full2plot = gf.ts.copy()
#                 var_full2plot.mask = gf.dc_mask
#                 clim = malib.calcperc(var_full2plot, (2,98))
#                 plot_array(var_full2plot, clim, [glac_str + ' Ts'], 'inferno', 'ts (degC)', close_fig=False)
            

            # ===== SURFACE TEMPERATURE FOR THINNEST DEBRIS (connected to terminus) =====
            outbins_df, z_bin_edges = gf.hist_plot(bin_width=debris_prms.mb_bin_size)   
            bin_idx_dc = np.where(outbins_df['dc_bin_count_valid'] > 0)[0]
            bin_idx_dif = list(bin_idx_dc[1:] - bin_idx_dc[:-1])
            if not(np.sum(bin_idx_dif) == len(bin_idx_dc)-1):
                idx_jumpinbins = bin_idx_dif.index(next(filter(lambda x: x>1, bin_idx_dif)))
                bin_idx_dc = bin_idx_dc[0:idx_jumpinbins+1]
            
            ts_min = np.nanmin(outbins_df.loc[bin_idx_dc,'dc_ts_med'].values)
            ts_max = np.nanmax(outbins_df.loc[bin_idx_dc,'dc_ts_med'].values)
            
            if debug:
                print('ts_min:', np.round(ts_min,1), ' ts_max:', np.round(ts_max,1))
            
            if np.isnan(ts_min) and np.isnan(ts_max):
                troubleshoot_fp = (debris_prms.output_fp + 'errors/no_Ts_data-extrap/' + debris_prms.roi + '/')
                if not os.path.exists(troubleshoot_fp):
                    os.makedirs(troubleshoot_fp)
                txt_fn_extrapfail = glac_str + "-noTs-extrap.txt"
                with open(troubleshoot_fp + txt_fn_extrapfail, "w") as text_file:
                    text_file.write(glac_str + ' no surface temperature data but made it past preprocessing to extrap')

            else:
                # ===== ESTIMATE DEBRIS THICKNESS FOR EACH GLACIER INDIVIDUALLY =====
                # Load parameters from nearest neighbor
                rgiid_nearest_list = nearest_dict[main_glac_rgi_missing.loc[glac_idx, 'rgino_str']]

                n_nearest = 0
                n_success = 0
                min_n_nearest = 10
                hd_ts_list = []
                mf_list = []
                while n_nearest < n_glac_nearest and n_success < min_n_nearest:
                    rgi_str_nearest = rgiid_nearest_list[n_nearest]
                    if rgi_str_nearest.startswith('0'):
                        rgi_str_nearest = rgi_str_nearest[1:]
#                     if debug:
#                         print(n_nearest, 'rgi nearest:', rgi_str_nearest)

                    # Load parameters
                    df_opt_fn = rgi_str_nearest + '_hdopt_prms.csv'
                    roi_nearest = str(int(rgi_str_nearest.split('.')[0])).zfill(2)
                    if roi_nearest in ['13','14','15']:
                        roi_nearest = 'HMA'
                    hdopt_prms_fp = debris_prms.output_fp + 'hd_opt_prms/' + roi_nearest + '/'
                    df_opt = pd.read_csv(hdopt_prms_fp + df_opt_fn)
                    melt_2cm = df_opt.loc[0,'melt_mwea_2cm']
                    melt_cleanice = df_opt.loc[0,'melt_mwea_clean']
                    func_coeff = [df_opt.loc[0,'b0'], df_opt.loc[0,'k']]
                    func_coeff_ts = [df_opt.loc[0,'a'], df_opt.loc[0,'b'], df_opt.loc[0,'c']]
                    
                    if melt_2cm_nearest is None:
                        melt_2cm_nearest = melt_2cm.copy()
                        melt_cleanice_nearest = melt_cleanice.copy()
                        func_coeff_nearest = func_coeff.copy()

                    # Estimate debris thickness of thinnest bin
                    hd_thin = debris_fromts_func(ts_min, func_coeff_ts[0], func_coeff_ts[1], func_coeff_ts[2])
                    hd_thick = debris_fromts_func(ts_max, func_coeff_ts[0], func_coeff_ts[1], func_coeff_ts[2])
                    
                    if debug:
                        print('     thin:', np.round(ts_min,1), np.round(hd_thin,3), 'm', 
                              '    thick:', np.round(ts_max,1), np.round(hd_thick,3), 'm')

    #                 print(func_coeff_ts[0], func_coeff_ts[1], func_coeff_ts[2], ts_min, hd_thin)

                    # Minimum and maximum debris thickness are reasonable
                    if hd_thin > 0 and hd_thin < 0.2 and hd_thick < debris_prms.hd_max:

                        if debug:
                            print('  ', n_nearest, 'hd thin:', np.round(hd_thin,2), 'hd thick:', np.round(hd_thick,2))

                        hd_array = debris_fromts_maskedarray(gf.ts, func_coeff_ts[0], func_coeff_ts[1], func_coeff_ts[2])
                        hd_array[hd_array>debris_prms.hd_max] = debris_prms.hd_max
                        hd_array[hd_array<0] = 0

                        hd_ts_list.append(hd_array)
                        n_success += 1

                    n_nearest += 1
                    
                if len(hd_ts_list) == 0:
                    # Record initial failure
                    troubleshoot_fp = (debris_prms.output_fp + 'errors/extrap_failed_rnd1/' + debris_prms.roi + '/')
                    if not os.path.exists(troubleshoot_fp):
                        os.makedirs(troubleshoot_fp)
                    txt_fn_extrapfail = glac_str + "-extrap_failed_rnd1.txt"
                    with open(troubleshoot_fp + txt_fn_extrapfail, "w") as text_file:
                        text_file.write(glac_str + ' failed to find any reasonable extrapolation estimates in first round')
                    
                    
                # ===== SECOND ROUND: NEGATIVE VALUES CAUSE HD_THIN TO BE 0 =====
                #  assume the absolute surface temperature is wrong, but spatial variation is representative
                #  of debris thickness variations, so increase surface temperature until find good fit
                if len(hd_ts_list) == 0 and ts_min < 0:
                    
                    if debug:
                        print('\n-----\nROUND 2 OF EXTRAPOLATION')
                    
                    ts_offset = abs(ts_min)
                    n_offset = 0
                    while len(hd_ts_list) < 5 and n_offset < 20:
                        gf.dc_ts = np.ma.array(gf.dc_ts.data.copy() + ts_offset, mask=gf.dc_ts.mask)
#                             ts_array = gf.ts.data.copy() + ts_offset
                        gf.ts = np.ma.array(gf.ts.data.copy() + ts_offset, mask=gf.ts.mask)
                        outbins_df, z_bin_edges = gf.hist_plot(bin_width=debris_prms.mb_bin_size)
                        ts_min = np.nanmin(outbins_df.loc[bin_idx_dc,'dc_ts_med'].values)
                        ts_max = np.nanmax(outbins_df.loc[bin_idx_dc,'dc_ts_med'].values)
                        # Estimate debris thickness of thinnest bin
                        hd_thin = debris_fromts_func(ts_min, func_coeff_ts[0], func_coeff_ts[1], func_coeff_ts[2])
                        hd_thick = debris_fromts_func(ts_max, func_coeff_ts[0], func_coeff_ts[1], func_coeff_ts[2])

                        if debug:
                            print('n_offset:', n_offset, ts_min, ts_max)

                        # ===== ESTIMATE DEBRIS THICKNESS FOR EACH GLACIER INDIVIDUALLY =====
                        # Load parameters from nearest neighbor
                        rgiid_nearest_list = nearest_dict[main_glac_rgi_missing.loc[glac_idx, 'rgino_str']]

                        n_nearest = 0
                        n_success = 0
                        min_n_nearest = 10
                        hd_ts_list = []
                        mf_list = []
                        while n_nearest < n_glac_nearest and n_success < min_n_nearest:
                            rgi_str_nearest = rgiid_nearest_list[n_nearest]
                            if rgi_str_nearest.startswith('0'):
                                rgi_str_nearest = rgi_str_nearest[1:]

                            # Load parameters
                            df_opt_fn = rgi_str_nearest + '_hdopt_prms.csv'
                            roi_nearest = str(int(rgi_str_nearest.split('.')[0])).zfill(2)
                            if roi_nearest in ['13','14','15']:
                                roi_nearest = 'HMA'
                            hdopt_prms_fp = debris_prms.output_fp + 'hd_opt_prms/' + roi_nearest + '/'
                            df_opt = pd.read_csv(hdopt_prms_fp + df_opt_fn)
                            melt_2cm = df_opt.loc[0,'melt_mwea_2cm']
                            melt_cleanice = df_opt.loc[0,'melt_mwea_clean']
                            func_coeff = [df_opt.loc[0,'b0'], df_opt.loc[0,'k']]
                            func_coeff_ts = [df_opt.loc[0,'a'], df_opt.loc[0,'b'], df_opt.loc[0,'c']]

                            # Estimate debris thickness of thinnest bin
                            hd_thin = debris_fromts_func(ts_min, func_coeff_ts[0], func_coeff_ts[1], func_coeff_ts[2])
                            hd_thick = debris_fromts_func(ts_max, func_coeff_ts[0], func_coeff_ts[1], func_coeff_ts[2])
                            # Minimum and maximum debris thickness are reasonable
                            if hd_thin > 0.01 and hd_thin < 0.2 and hd_thick < debris_prms.hd_max:

                                if debug:
                                    print('  ', n_nearest, 'hd thin:', np.round(hd_thin,2), 'hd thick:', np.round(hd_thick,2))

                                hd_array = debris_fromts_maskedarray(gf.ts, func_coeff_ts[0], func_coeff_ts[1], func_coeff_ts[2])
                                hd_array[hd_array>debris_prms.hd_max] = debris_prms.hd_max
                                hd_array[hd_array<0] = 0

                                hd_ts_list.append(hd_array)
                                n_success += 1

                            n_nearest += 1

                        n_offset += 1
                        ts_offset = 1
                        
                        
                if len(hd_ts_list) == 0:
                    # Record initial failure
                    troubleshoot_fp = (debris_prms.output_fp + 'errors/extrap_failed_rnd2/' + debris_prms.roi + '/')
                    if not os.path.exists(troubleshoot_fp):
                        os.makedirs(troubleshoot_fp)
                    txt_fn_extrapfail = glac_str + "-extrap_failed_rnd2.txt"
                    with open(troubleshoot_fp + txt_fn_extrapfail, "w") as text_file:
                        text_file.write(glac_str + ' failed to find any reasonable extrapolation estimates in second round')
                    
                # ===== THIRD ROUND: ASSUME TOO POSITIVE CAUSING HD TO BE VERY THICK =====
                #  assume the absolute surface temperature is wrong, but spatial variation is representative
                #  of debris thickness variations, so increase surface temperature until find good fit
                if len(hd_ts_list) == 0 and ts_max > 20:
                    
                    if debug:
                        print('\n-----\nROUND 3 OF EXTRAPOLATION')
                    
                    ts_offset = -1
                    n_offset = 0
                    while len(hd_ts_list) < 5 and n_offset < 20:
                        gf.dc_ts = np.ma.array(gf.dc_ts.data.copy() + ts_offset, mask=gf.dc_ts.mask)
                        gf.ts = np.ma.array(gf.ts.data.copy() + ts_offset, mask=gf.ts.mask)
                        outbins_df, z_bin_edges = gf.hist_plot(bin_width=debris_prms.mb_bin_size)
                        ts_min = np.nanmin(outbins_df.loc[bin_idx_dc,'dc_ts_med'].values)
                        ts_max = np.nanmax(outbins_df.loc[bin_idx_dc,'dc_ts_med'].values)
                        # Estimate debris thickness of thinnest bin
                        hd_thin = debris_fromts_func(ts_min, func_coeff_ts[0], func_coeff_ts[1], func_coeff_ts[2])
                        hd_thick = debris_fromts_func(ts_max, func_coeff_ts[0], func_coeff_ts[1], func_coeff_ts[2])

                        if debug:
                            print('n_offset:', n_offset, ts_min, ts_max)

                        # ===== ESTIMATE DEBRIS THICKNESS FOR EACH GLACIER INDIVIDUALLY =====
                        # Load parameters from nearest neighbor
                        rgiid_nearest_list = nearest_dict[main_glac_rgi_missing.loc[glac_idx, 'rgino_str']]

                        n_nearest = 0
                        n_success = 0
                        min_n_nearest = 10
                        hd_ts_list = []
                        mf_list = []
                        while n_nearest < n_glac_nearest and n_success < min_n_nearest:
                            rgi_str_nearest = rgiid_nearest_list[n_nearest]
                            if rgi_str_nearest.startswith('0'):
                                rgi_str_nearest = rgi_str_nearest[1:]

                            # Load parameters
                            df_opt_fn = rgi_str_nearest + '_hdopt_prms.csv'
                            roi_nearest = str(int(rgi_str_nearest.split('.')[0])).zfill(2)
                            if roi_nearest in ['13','14','15']:
                                roi_nearest = 'HMA'
                            hdopt_prms_fp = debris_prms.output_fp + 'hd_opt_prms/' + roi_nearest + '/'
                            df_opt = pd.read_csv(hdopt_prms_fp + df_opt_fn)
                            melt_2cm = df_opt.loc[0,'melt_mwea_2cm']
                            melt_cleanice = df_opt.loc[0,'melt_mwea_clean']
                            func_coeff = [df_opt.loc[0,'b0'], df_opt.loc[0,'k']]
                            func_coeff_ts = [df_opt.loc[0,'a'], df_opt.loc[0,'b'], df_opt.loc[0,'c']]

                            # Estimate debris thickness of thinnest bin
                            hd_thin = debris_fromts_func(ts_min, func_coeff_ts[0], func_coeff_ts[1], func_coeff_ts[2])
                            hd_thick = debris_fromts_func(ts_max, func_coeff_ts[0], func_coeff_ts[1], func_coeff_ts[2])
                            # Minimum and maximum debris thickness are reasonable
                            if hd_thin > 0.01 and hd_thin < 0.2 and hd_thick < debris_prms.hd_max:

                                if debug:
                                    print('  ', n_nearest, 'hd thin:', np.round(hd_thin,2), 'hd thick:', np.round(hd_thick,2))

                                hd_array = debris_fromts_maskedarray(gf.ts, func_coeff_ts[0], func_coeff_ts[1], func_coeff_ts[2])
                                hd_array[hd_array>debris_prms.hd_max] = debris_prms.hd_max
                                hd_array[hd_array<0] = 0

                                hd_ts_list.append(hd_array)
                                n_success += 1

                            n_nearest += 1

                        n_offset += 1

                # ===== ESTIMATE DEBRIS THICKNESS FROM ALL COMBINATIONS =====
                if len(hd_ts_list) > 0:
                    # DEBRIS THICKNESS based on median of the plausible nearest values
                    hd_ts_all = np.array(hd_ts_list)
                    hd_ts_med = np.median(hd_ts_all, axis=0)
                    gf.debris_thick_ts = np.ma.array(hd_ts_med, mask=gf.dc_mask)

                    if debug:
                        close_fig=False
                    else:
                        close_fig=True

                    # Debris thickness
                    var_full2plot = gf.debris_thick_ts.copy()
                    clim = (0,1)
                    plot_array(var_full2plot, clim, [gf.glacnum + ' hd (from ts)'], 'inferno', 'hd (m)', 
                               fn=fig_extrap + gf.feat_fn +'_hd_ts.png', close_fig=close_fig)

                    # ===== EXPORT DEBRIS THICKNESS AND MELT FACTOR TIFS ===== 
                    # Debris thickness
                    gf.debris_thick_ts.mask = gf.dc_mask
                    iolib.writeGTiff(gf.debris_thick_ts, hd_fp + hd_fn, gf.ds_dict['z1'])

                    # Optimized parameters from nearest glacier for melt factor
                    hdopt_cns = ['glac_str', 'melt_mwea_clean', 'melt_mwea_2cm', 'b0', 'k']
                    df_hdopt_prms = pd.DataFrame(np.zeros((1,len(hdopt_cns))), columns=hdopt_cns)
                    df_hdopt_prms['glac_str'] = glac_str
                    df_hdopt_prms.loc[0,'melt_mwea_clean'] = melt_cleanice_nearest
                    df_hdopt_prms.loc[0,'melt_mwea_2cm'] = melt_2cm_nearest
                    df_hdopt_prms.loc[0,'b0'] = func_coeff_nearest[0]
                    df_hdopt_prms.loc[0,'k'] = func_coeff_nearest[1]
                    hdopt_prms_fp = debris_prms.output_fp + 'hd_opt_prms/' + debris_prms.roi + '/_extrap/'
                    if not os.path.exists(hdopt_prms_fp):
                        os.makedirs(hdopt_prms_fp)
                    df_hdopt_prms.to_csv(hdopt_prms_fp + glac_str + '_hdopt_prms_extrap.csv', index=False)
                    
                    # Melt factor
                    gf.meltfactor_ts = (
                        melt_fromdebris_func(gf.debris_thick_ts, func_coeff_nearest[0], func_coeff_nearest[1]) 
                        / melt_cleanice_nearest)
                    # limit melt rates to modeled 2 cm rate
                    gf.meltfactor_ts = np.ma.array(
                        maskedarray_gt(gf.meltfactor_ts, melt_2cm_nearest / melt_cleanice_nearest), 
                        mask=np.ma.getmask(gf.debris_thick_ts))
                    # Linearly interpolate between 0 cm and 2 cm for the melt rate
                    def meltfactor_0to2cm_adjustment(mf, melt_clean, melt_2cm, hd):
                        """ Linearly interpolate melt factors between 0 and 2 cm 
                            based on clean ice and 2 cm sub-debris melt """
                        mf = np.nan_to_num(mf,0)
                        mf[(hd >= 0) & (hd < 0.02)] = (
                            1 + hd[(hd >= 0) & (hd < 0.02)] / 0.02 * (melt_2cm - melt_clean) / melt_clean)
                        return mf
                    gf.meltfactor_ts = np.ma.array(
                        meltfactor_0to2cm_adjustment(gf.meltfactor_ts.data.copy(), melt_cleanice_nearest, 
                                                     melt_2cm_nearest, gf.debris_thick_ts.data), 
                        mask=np.ma.getmask(gf.debris_thick_ts))
                    # Plot melt factor
                    var_full2plot = gf.meltfactor_ts.copy()
                    clim = (0,1.25)
                    plot_array(var_full2plot, clim, [gf.glacnum + ' melt factor'], 'inferno', 'melt factor (-)', 
                               fn=fig_extrap + gf.feat_fn +'_mf.png', close_fig=True)

                    gf.meltfactor_ts.mask = gf.dc_mask
                    mf_fn = debris_prms.mf_fn_sample.replace('XXXX',gf.glacnum).replace('.tif','_extrap.tif')
                    iolib.writeGTiff(gf.meltfactor_ts, mf_fp + mf_fn, gf.ds_dict['z1'])


                    # ===== EXPORT THE BINNED DEBRIS THICKNESS AND MELT FACTOR =====
                    # Output debris thickness
                    outbins_df, z_bin_edges = gf.hist_plot(bin_width=debris_prms.mb_bin_size)
                    hd_extrap_bin_fp = debris_prms.mb_binned_fp_wdebris_hdts + '../_wdebris_hdts_extrap/'
                    if not os.path.exists(hd_extrap_bin_fp):
                        os.makedirs(hd_extrap_bin_fp)
                    outbins_df.to_csv(hd_extrap_bin_fp + glac_str + '_mb_bins_hdts_extrap.csv', index=False)

                else:
                    troubleshoot_fp = (debris_prms.output_fp + 'errors/extrap_failed/' + debris_prms.roi + '/')
                    if not os.path.exists(troubleshoot_fp):
                        os.makedirs(troubleshoot_fp)
                    txt_fn_extrapfail = glac_str + "-extrap_failed.txt"
                    with open(troubleshoot_fp + txt_fn_extrapfail, "w") as text_file:
                        text_file.write(glac_str + ' failed to find any reasonable extrapolation estimates')

0 17.00001
500 17.02367
1000 17.03795
1500 17.05132
2000 17.06066
2500 17.07111
3000 17.09021
3500 17.10405
4000 17.11827
4500 17.13056
5000 17.13839
5500 17.15088
4290 17.12709
4291 17.12711
4292 17.12712
4293 17.12713
4294 17.12718
4295 17.12719
4296 17.12720
4297 17.12721
4298 17.12724
4299 17.12725
4300 17.12726
4301 17.12728
4302 17.12729
4303 17.12731
4304 17.12732
4305 17.12735
4306 17.12739
4307 17.12740
4308 17.12746
4309 17.12751
4310 17.12759
4311 17.12760
4312 17.12761
4313 17.12766
4314 17.12769
4315 17.12773
4316 17.12774
4317 17.12775
4318 17.12777
4319 17.12778
4320 17.12779
4321 17.12780
4322 17.12781
4323 17.12782
4324 17.12787
4325 17.12789
4326 17.12792
4327 17.12793
4328 17.12795
4329 17.12799
4330 17.12801
4331 17.12802
4332 17.12809
4333 17.12811
4334 17.12816
4335 17.12820
4336 17.12821
4337 17.12822
4338 17.12823
4339 17.12824
4340 17.12825
4341 17.12828
4342 17.12837
4343 17.12839
4344 17.12840
4345 17.12842
4346 17.12843
4347 17.12845
4348 17.12846
4349 17.12

4864 17.13509
4865 17.13511
4866 17.13512
4867 17.13513
4868 17.13514
4869 17.13515
4870 17.13516
4871 17.13517
4872 17.13518
4873 17.13519
4874 17.13520
4875 17.13521
4876 17.13522
4877 17.13524
4878 17.13525
4879 17.13527
4880 17.13528
4881 17.13529
4882 17.13530
4883 17.13531
4884 17.13532
4885 17.13533
4886 17.13534
4887 17.13535
4888 17.13536
4889 17.13538
4890 17.13539
4891 17.13540
4892 17.13541
4893 17.13543
4894 17.13544
4895 17.13545
4896 17.13547
4897 17.13548
4898 17.13549
4899 17.13550
4900 17.13551
4901 17.13552
4902 17.13553
4903 17.13554
4904 17.13555
4905 17.13556
4906 17.13557
4907 17.13558
4908 17.13559
4909 17.13560
4910 17.13561
4911 17.13563
4912 17.13566
4913 17.13569
4914 17.13573
4915 17.13574
4916 17.13575
4917 17.13576
4918 17.13578
4919 17.13579
4920 17.13582
4921 17.13583
4922 17.13584
4923 17.13586
4924 17.13587
4925 17.13588
4926 17.13589
4927 17.13591
4928 17.13592
4929 17.13593
4930 17.13595
4931 17.13596
4932 17.13597
4933 17.13598
4934 17.13599
4935 1

5450 17.14987
5451 17.14988
5452 17.14994
5453 17.14995
5454 17.15000
5455 17.15001
5456 17.15003
5457 17.15004
5458 17.15005
5459 17.15007
5460 17.15008
5461 17.15009
5462 17.15011
5463 17.15013
5464 17.15016
5465 17.15020
5466 17.15021
5467 17.15022
5468 17.15024
5469 17.15026
5470 17.15030
5471 17.15032
5472 17.15035
5473 17.15037
5474 17.15038
5475 17.15039
5476 17.15043
5477 17.15046
5478 17.15047
5479 17.15049
5480 17.15051
5481 17.15055
5482 17.15056
5483 17.15057
5484 17.15058
5485 17.15059
5486 17.15063
5487 17.15064
5488 17.15065
5489 17.15066
5490 17.15069
5491 17.15071
5492 17.15073
5493 17.15074
5494 17.15078
5495 17.15080
5496 17.15081
5497 17.15083
5498 17.15086
5499 17.15087
5500 17.15088
5501 17.15089
5502 17.15090
5503 17.15091
5504 17.15092
5505 17.15093
5506 17.15094
5507 17.15095
5508 17.15098
5509 17.15099
5510 17.15100
5511 17.15101
5512 17.15104
5513 17.15107
5514 17.15108
5515 17.15109
5516 17.15110
5517 17.15111
5518 17.15113
5519 17.15114
5520 17.15116
5521 1

In [ ]:
# bin_idx_dc_thin = bin_idx_dc[-1]
# outbins_df.loc[bin_idx_dc_thin, 'hd_ts_med_m']
# outbins_df.loc[:,['bin_center_elev_m', 'dc_ts_med', 'hd_ts_med_m']]

In [ ]:
print('\nDONE!\n')

In [ ]:
print('TO-DO LIST:')
print('  - Mosaic extrapolated and calibrated glaciers into 1-degree pixels')
print('  - Test robustness on glaciers with data (Ngozumpa, Miage, etc.)')